In [ ]:
# !pip install -q pytorch-lightning
!pip install -q --upgrade transformers datasets rouge_score evaluate
!pip install -q wandb

In [ ]:
!nvidia-smi

Sat Jan 11 21:35:01 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#datasets.__version__

In [ ]:
from evaluate import load
#rouge_metric = load('rouge')

In [ ]:
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
# import pytorch_lightning as pl
import torch
# from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse
import datetime

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from IPython.display import display, HTML
import random
import nltk  # Here to have a nice missing dependency error message early on
import numpy as np
from datasets import load_dataset#, load_metric

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

nltk.download('punkt')

import wandb
wandb.login()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### References: [Official](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/summarization.ipynb#scrollTo=vc0BSBLIIrJQ), [Community](https://colab.research.google.com/github/elsanns/xai-nlp-notebooks/blob/master/fine_tune_bart_summarization_two_langs.ipynb#scrollTo=6R9d7ELIpX9F)

In [ ]:
BASE_PATH = "/content/drive/MyDrive/DSAIT4090_FinalProject/code/claim-decomp/"

MODEL_PATH = "google/flan-t5-base"
TOKENIZER_PATH = "google/flan-t5-base"
SAVE_PATH = BASE_PATH + "test/"
LOGGING_PATH = BASE_PATH + "test_logs/"
SAVE_MODEL_PATH = BASE_PATH + "test:model/"

In [ ]:
import pandas as pd
train = pd.read_csv(BASE_PATH + "train.csv")
val = pd.read_csv(BASE_PATH + "val.csv")
test = pd.read_csv(BASE_PATH + "test.csv")
train

,Unnamed: 0,claim,subquestions
0,0,"""Unemployment is low because everyone has two ...",Can low unemployment rates be attributed to ev...
1,1,Says the impeachment of President Donald Trump...,Have any presidents who have been impeached in...
2,2,"""When I withdrew in June of 2008, polls were s...",Did Obama lose this level of potential support...
3,3,Text from someone “connected” to Raleigh polic...,Did this information come from the police of t...
4,4,"“Some states, like Montana and Nebraska, are g...","Are some states getting $300,000 in federal st..."
...,...,...,...
795,795,Says Miami-Dade Mayor Carlos Gimenez “approved...,Did Gimenez give himself these benefits?\nHas ...
796,796,"""Classified material has a header which says ‘...",Were the emails sent to Hillary Clinton marked...
797,797,The border surge is “challenging” because the ...,Did the Trump administration make major change...
798,798,"We ""have laws on the books designed to prevent...",Do any laws exist that prevent the mentally il...


In [ ]:
# data_subq["subquestions"] = data_subq["subquestions"].apply(lambda x: x.replace("[","").replace("]",""))
# data_subq

In [ ]:
# from sklearn.model_selection import train_test_split
# train,val = train_test_split(data_subq,test_size=0.10,random_state=42)
# train

,question,answer,subquestions
1056,Did Pink Floyd have a song about the French Ri...,True,Question 0: What songs by Pink Floyd are named...
2085,Did Moon Jae-in's residence exist when the Wor...,False,Question 0: Where does Moon Jae-in live?\nAnsw...
637,Did the death of Helen Palmer have a significa...,True,Question 0: What relatives did Helen Palmer ha...
48,Is Menthol associated with Thanksgiving?,False,Question 0: What are some common dishes served...
509,Is Capricorn the hypothetical zodiac sign of S...,True,Question 0: What animal represents the zodiac ...
...,...,...,...
1638,Is Cookie Monster's diet Paleo friendly?,False,Question 0: What is the major component of the...
1095,Is LG Electronics located in a city with an of...,True,Question 0: Which city is LG Electronics headq...
1130,Would it be difficult to snowboard on Venus?,True,Question 0: What kind of surface is suitable f...
1294,Can a minor replicate the double-slit experiment?,True,Question 0: What species of living things does...


In [ ]:
# train.to_csv("train_strategyq_with_answer.csv",index=False)
# val.to_csv("val_strategyq_with_answer.csv",index=False)

# Loading and processing the data

In [ ]:
TRAIN_FILE_PATH = BASE_PATH + "train.csv"
VAL_FILE_PATH = BASE_PATH + "val.csv"
TEST_FILE_PATH = BASE_PATH + "test.csv"

In [ ]:
data_files = {}
data_files["train"] = TRAIN_FILE_PATH
data_files["validation"] = VAL_FILE_PATH
data_files["test"] = TEST_FILE_PATH

In [ ]:
xmetric = load('rouge')#load_metric("rouge")
data_files
raw_datasets = load_dataset("csv", data_files=data_files)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
metric = load('rouge')#load_metric("rouge")


In [ ]:
raw_datasets["train"][0]

{'Unnamed: 0': 0,
 'claim': '"Unemployment is low because everyone has two jobs. Unemployment is low because people are working 60, 70, 80 hours a week and can barely feed their family."',
 'subquestions': 'Can low unemployment rates be attributed to everyone having 2 jobs?\nIs unemployment even currently low because of factors stated by Ocasio Cortez?\nCan low unemployment be attributed to long work hours?'}

In [ ]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,Unnamed: 0,claim,subquestions
0,348,"Italy “went against the WHO's ‘law’” against COVID-19 autopsies and “discovered covid is not a virus, but a bacterium” that “clots the blood and reduces the oxygen saturation.”","Did Italy go against WHO's guidelines and prefer an ""illegal"" autopsy?\nDid Italy discover COVID to be an easily treatable bacteria because of conducting an illegal autopsy?\nWas COVID discovered to be an easily treatable bacteria?"
1,498,"""In 2004, George W. Bush lifted the ban on those (AR-15 type) rifles, and since then, the number of massacres has increased 183%. During the ban, there were 12 incidents. Since he lifted the ban, there were 34 incidents.""",Did George Bush lift the ban on assault rifles?\nHas the incidence of mass shootings increased dramatically since the ban was lifted?
2,223,"Says Pat Robertson said, ""We must impose a rule on the gay population that would require them to wear specially-colored clothes.""","Did Robertson in general make any statements about the gay population and/or their clothing requirements?\nDid Pat Robertson explicitly say "" We must impose a rule on the gay population that would require them to wear specially - colored clothes . ""\nIs there proof Pat Robertson said "" We must impose a rule on the gay population that would require them to wear specially - colored clothes . “?"
3,1,"Says the impeachment of President Donald Trump would be the first in U.S. history ""without a specific criminal statute or crime that the president who is impeached would have allegedly committed.""","Have any presidents who have been impeached in the past charged with violating a criminal statute?\nIf Trump is impeached without being charged with violating a criminal statute, would that be the first instance of it in U.S. history?\nWere the two presidents who were impeached accused of any crimes?"
4,251,Wisconsin was the last state to start paying COVID-related federal unemployment benefits.,Was Wisconsin the last state to start paying covid unemployment benefits?



- Classify type of claim

For each type of claim
- Decompose claim
- Get claim prediction

## Tokenization and Dataset Preparation

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-base', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>'

In [ ]:
max_input_length = 64
max_target_length = 128

In [ ]:

def preprocess_function(examples, prefix="decompose the compositional question:"):
    inputs = [prefix + doc for doc in examples["claim"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # The "labels" are the tokenized outputs:
    print(examples["subquestions"])
    labels = tokenizer(text_target=examples["subquestions"], max_length=max_target_length, truncation=True)
    print(labels)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=raw_datasets['train'].column_names)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

['Can low unemployment rates be attributed to everyone having 2 jobs?\nIs unemployment even currently low because of factors stated by Ocasio Cortez?\nCan low unemployment be attributed to long work hours?', 'Have any presidents who have been impeached in the past charged with violating a criminal statute?\nIf Trump is impeached without being charged with violating a criminal statute, would that be the first instance of it in U.S. history?\nWere the two presidents who were impeached accused of any crimes?', 'Did Obama lose this level of potential support from pro-Clinton voters?\nDid some Democrats not support Obama in the election?\nDid these polls verbatim ask if these voters would support Obama?', 'Did this information come from the police of this city?\nHas antifa stormed into wealthy neighborhoods in any other cities?\nHas antifa stormed into wealthy neighborhoods in this city?', 'Are some states getting $300,000 in federal stimulus money per each COVID-19 case?\nIs New York getti

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

['Is voting fraud widespread in the US?\nIs there a greater risk of voting fraud with mail-in ballots?\nIs there evidence of thousands of people committing mail-in voting fraud?', 'Was the federal aid given by Trump for hurricane Michael a historic commitment?\nDid Trump extend the federal aid for hurricane Michael?\nDid Ron DeSantis ask the Trump administration for a federal aid extension for hurricane Michael?', 'Is this ban directly linked to outbreaks of food-borne illnesses?\nIs there a correlation between this ban and outbreaks of food-borne illnesses?\nIs the study that is referenced by Quintero based on sound methodology?', 'Is it true that  The United States " decided waterboarding was torture back when we court - martialed American soldiers for waterboarding Philippine insurgents in the Philippine revolution . "?', 'Has Trump been accused of walking into a dressing room while contestants were changing?\nHas Trump been convicted of walking into a dressing room while minors wer

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

["Has Barr received $36,550 from payday lenders?\nDid Barr vote for legislation that would weaken restrictions for payday lenders?\nAre there any protections for service members using payday lending services?\nHas Barr's voting record directly affected protection for veterans against payday lenders?", 'Did Nicholson make $1 million consulting for the plants that laid off 2,000 people?', 'Will people be taken into local police custody if they do not respond to this call?\nDo people that receive these calls have serious allegations against them?\nDo these calls legitimately represent local police?', 'Was Donald Trump "Excited" for the 2008 housing crash?\nDid Trump use the housing crash to obtain cheap real estate?', 'Has Obama cited climate change as the top national security threat?\nHas Obama prioritized other concerns over climate change?', 'Did Kamala Harris say she would sign an executive order for law enforcement to collect guns?', "Did a Kenosha car dealer kill himself because in

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

# Building the model

## Metrics

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(predictions[0])
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    print("decoded gen",decoded_preds)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value for key, value in result.items()}#{key: value['f1'] * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Loading the model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

## Training Args

In [ ]:
epochs = 36
batch_size = 8
lr = 2e-5

In [ ]:
!pip install accelerate -U

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir=SAVE_PATH,
    learning_rate=lr,
    do_train = True,
    do_eval = True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    num_train_epochs=epochs,
    predict_with_generate=True,
    generation_max_length=512,
    logging_dir=LOGGING_PATH,
    logging_steps=300,
    save_steps=300,
    report_to = "wandb",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
wandb_run = wandb.init(
    project="flant5_subq",
    config={
        "per_device_train_batch_size": batch_size,
        "learning_rate": lr})

now = datetime.datetime.now()
current_time = now.strftime("%m/%d/%Y, %H:%M:%S")
wandb_run.name = "run_" + current_time

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

<ipython-input-38-82a9305c052e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# Training

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
trainer.evaluate() # Evaluation before fine-tuning

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


[    0   438 10601    57  4842     6   105  4188   129  2909    11  2909
    13   151  3823    16 10843    31     7   840   562     6  8097 16524
     7    66   147     8   286  1239     1     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


{'eval_loss': 2.5070786476135254,
 'eval_model_preparation_time': 0.0199,
 'eval_rouge1': 0.3392,
 'eval_rouge2': 0.1707,
 'eval_rougeL': 0.2858,
 'eval_rougeLsum': 0.3105,
 'eval_gen_len': 57.6,
 'eval_runtime': 164.9425,
 'eval_samples_per_second': 1.213,
 'eval_steps_per_second': 0.152}

In [ ]:
val

,Unnamed: 0,claim,subquestions
0,0,"With voting by mail, “you get thousands and th...",Is voting fraud widespread in the US?\nIs ther...
1,1,"""I’ve already traveled to Washington, D.C., an...",Was the federal aid given by Trump for hurrica...
2,2,Says that when San Francisco banned plastic gr...,Is this ban directly linked to outbreaks of fo...
3,3,"The United States ""decided waterboarding was t...","Is it true that The United States "" decided w..."
4,4,"Quotes Donald Trump as saying, “I’ll tell you ...",Has Trump been accused of walking into a dress...
...,...,...,...
195,195,“So now if you don’t comply” and get a COVID-1...,Is there a requirement to get a covid vaccinat...
196,196,Says the toddler shown in a photo clinging to ...,Is it true that the toddler shown in the photo...
197,197,"Says one year later, ""the coalition to defeat ...",Is ISIS currently still losing territory?\nDid...
198,198,"“In 2018, the providers the state paid through...",Did the ACO reduce the amount spent on healthc...


In [ ]:
# %%wandb

trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
300,2.122300,1.816834,0.398000,0.202800,0.354200,0.357200,39.500000
600,1.893800,1.795213,0.404300,0.206700,0.356800,0.368100,39.100000
900,1.770400,1.786664,0.408500,0.209700,0.359500,0.371500,43.905000
1200,1.680200,1.786132,0.407000,0.209900,0.358000,0.371200,42.335000
1500,1.605700,1.791047,0.414800,0.212100,0.362800,0.379400,40.355000
1800,1.541400,1.799002,0.420200,0.219600,0.368100,0.386400,39.095000
2100,1.494500,1.809132,0.419500,0.217000,0.364800,0.386000,39.575000
2400,1.457200,1.813546,0.419000,0.217900,0.366500,0.385600,40.020000
2700,1.419600,1.821409,0.414600,0.213000,0.361300,0.380300,40.170000
3000,1.403000,1.826143,0.409900,0.211900,0.359600,0.376500,40.470000


[    0  3963     8 10601    57  4842   358    43     3     9  2841  1504
    30     8 10601   433    58  3963     8 10601    57  4842   358    43
     3     9  2841  1113    30     8 10601   433    58     1     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100]
decoded gen ['Did the voting by mail system have a negative effect on the voting process? Did the voting by mail system have a negative impact on the voting process?', "Did Trump's administration provide assistance to the communities that Michael battered? Did Trump's administration provide assistance to the communities that Michael battered?", 'Did San Francisco ban plastic grocery bags? Did San Francisco ban plastic grocery bags? Did

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=3600, training_loss=1.5957328287760417, metrics={'train_runtime': 2096.4578, 'train_samples_per_second': 13.737, 'train_steps_per_second': 1.717, 'total_flos': 2397035909578752.0, 'train_loss': 1.5957328287760417, 'epoch': 36.0})

In [ ]:
trainer.evaluate()

[    0    27     7    34  1176    24 10601    57  4842    19     3     9
   194    12   129  2909    11  2909    13   151  3823    16 10843    31
     7   840   562  8097 16524     7    66   147     8   286    58     1
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

{'eval_loss': 1.7861316204071045,
 'eval_rouge1': 0.407,
 'eval_rouge2': 0.2099,
 'eval_rougeL': 0.358,
 'eval_rougeLsum': 0.3712,
 'eval_gen_len': 42.335,
 'eval_runtime': 49.74,
 'eval_samples_per_second': 4.021,
 'eval_steps_per_second': 0.503,
 'epoch': 36.0}

In [ ]:
trainer.save_model(SAVE_MODEL_PATH)

In [ ]:
wandb_run.finish()

eval/gen_len,▂▁█▆▃▁▂▂▃▃▂▃▆
eval/loss,▆▂▁▁▂▃▅▅▇███▁
eval/rouge1,▁▃▄▄▆███▆▅▅▅▄
eval/rouge2,▁▃▄▄▅█▇▇▅▅▅▄▄
eval/rougeL,▁▂▄▃▅█▆▇▅▄▄▃▃
eval/rougeLsum,▁▄▄▄▆███▇▆▆▅▄
eval/runtime,▁▁▆█▃▁▁▁▂▃▃▁▅
eval/samples_per_second,██▂▁▅█▇█▇▅▅█▃
eval/steps_per_second,██▂▁▅███▇▅▅█▃
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇████


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(SAVE_MODEL_PATH)

In [ ]:
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
model.device

device(type='cuda', index=0)

# Get QuanTemp Subquestions

In [ ]:
def generate_output(test_samples, model):
    inputs = tokenizer(
        test_samples,
        max_length=128,
        return_tensors="pt")

    input_ids = inputs.input_ids.to(model.device)
    #print(input_ids)
    #print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask,min_length = 64, max_length = 128, do_sample=True, top_p=0.95, top_k=50)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(test_samples)
    return output_str

In [ ]:
import json
# Make sure to load in the csv datasets from the Quantemp repo
# with open("train_claimdecomp_evidence_question_mapping.json") as f:
#   train_json = json.load(f)
# with open("val_claimdecomp_evidence_question_mapping.json") as f:
#   val_json = json.load(f)
with open("test_claims_quantemp.json") as f:
  test_json = json.load(f)
# train_json[0]

In [ ]:
# Test on one sample
print("decompose the compositional question:"+train_json[0]['claim'], sep="\n", end="\n\n")
generate_output("decompose the compositional question:"+train_json[0]['claim'], model)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


decompose the compositional question:In her budget speech, Nirmala Sitharaman claimed that the Government distributed 35,000 crore LED bulbs in the country.



['Is the Government providing 35,000 crore LED bulbs in the country? Was the Government offering 35,000 crore LED bulbs in the country? Was the Government offering 35,000 crore LED bulbs in the country? Did Sitharaman claim the Government had distributed 35,000 crore LED bulbs? Did Sitharaman claim the Government had distributed 35,000 crore LED bulbs in the country?']

In [ ]:
quantemp_train_data = []
for sample in train_json:
  output = generate_output("decompose the compositional question:"+sample['claim'], model)
  quantemp_train_data.append({'claim': sample['claim'], 'subquestions': output})

In [ ]:
with open("train_flant5_claimdecomp_subquestions.json", "w") as f:
  json.dump(quantemp_train_data, f)

In [ ]:
!cp  /content/train_flant5_claimdecomp_subquestions.json /content/drive/MyDrive/DSAIT4090_FinalProject/code/claim-decomp

In [ ]:
quantemp_val_data = []
for sample in val_json:
  output = generate_output("decompose the compositional question:"+sample['claim'], model)
  quantemp_val_data.append({'claim': sample['claim'], 'subquestions': output})

[{'claim': 'In her budget speech, Nirmala Sitharaman claimed that the Government distributed 35,000 crore LED bulbs in the country.',
  'subquestions': ['Did Sitharaman claim 35,000 crore LED bulbs were distributed in the country? Did Sitharaman claim 35,000 crore LED bulbs were distributed? Were all the bulbs distributed in the country by Sitharaman? Did Sitharaman claim the Government distribute 35,000 crore LED bulbs in the country?']}]

In [ ]:
with open("val_flant5_claimdecomp_subquestions.json", "w") as f:
  json.dump(quantemp_val_data, f)

In [ ]:
!cp  /content/val_flant5_claimdecomp_subquestions.json /content/drive/MyDrive/DSAIT4090_FinalProject/code/claim-decomp

In [ ]:
quantemp_test_data = []
for sample in test_json:
  output = generate_output("decompose the compositional question:"+sample['claim'], model)
  quantemp_test_data.append({'claim': sample['claim'], 'subquestions': output})

decompose the compositional question:"The non-partisan Congressional Budget Office concluded ObamaCare will cost the U.S. more than 800,000 jobs."
decompose the compositional question:"More than 50 percent of immigrants from (El Salvador, Guatemala and Honduras) use at least one major welfare program once they get here."
decompose the compositional question:UK government banned Covid vaccine for children age 5-11
decompose the compositional question:"[In 2014-2015] coverage for the rotavirus vaccine exceeded the 95% target and the pneumococcal vaccine reached 91.5%."
decompose the compositional question:In September 2021, the U.K. government announced its intention to create a new criminal offense of pet abduction.
decompose the compositional question:Labour took £1.5 million from Just Stop Oil.
decompose the compositional question:McDonald's has announced that everyone who shares this link will receive 2 FREE McFamily box
decompose the compositional question:"NASA just announced a 100

In [ ]:
with open("test_flant5_claimdecomp_subquestions.json", "w") as f:
  json.dump(quantemp_test_data, f)

In [ ]:
!cp  /content/test_flant5_claimdecomp_subquestions.json /content/drive/MyDrive/DSAIT4090_FinalProject/code/claim-decomp